In [9]:
# imports
import os
import io
import tokenize
import numpy as np
import pandas as pd

import re


In [22]:
# paths
# zeppelin_folder = '/home/ourownstory/Documents/SOL/data/Zeppelin/Zeppelin/crowdsale/'
zeppelin_joined = '/home/ourownstory/Documents/SOL/derived/joined/Zeppelin/Zeppelin/'
# os.listdir(zeppelin_folder)
os.listdir(zeppelin_joined)
crowdsale_joined = 'crowdsale_Crowdsale.sol'
# BasicToken.sol     CappedToken.sol    ERC20Basic.sol  MintableToken.sol  SafeERC20.sol      TokenTimelock.sol
# BurnableToken.sol  DetailedERC20.sol  ERC20.sol       PausableToken.sol  StandardToken.sol  TokenVesting.sol

In [23]:
# src reader from dat_wrangling notebook
# NOTE: modified to return list

# strips pragma statements and comments
def read_src_nocomments(file_name, return_only_comments=False):
    with open(file_name, 'r') as f:
        src_list = []
        comments_list = []
        for line in f.readlines():
            # skip pragma
            if re.match('pragma solidity .*;', line.strip()):
                continue
            # single or multiline comments
            if re.match('/', line.strip()) or re.match('\*', line.strip()):
                comments_list.append(line)
                continue
            # inline comments
            if re.search('//', line):
                inline_comment = re.findall('[\s]*//.*\n', line)
                comments_list.extend(inline_comment)
                line = re.sub('[\s]*//.*\n', '\n', line)
                continue
                
            # add current src line
            src_list.append(line)
        
        if return_only_comments:
            return comments_list
        else:
            return src_list

def remove_import_lines(src):
#     print src
    regex = "import .*;"
    src = [x for x in src if not re.match(regex, x)]
#     print src
    return src   

In [30]:
# read in as txt
raw_lines = []
with open(zeppelin_joined+crowdsale_joined, 'r') as file:
    for line in file:
        raw_lines.append(line)
        
# read stripped src
# raw_lines = read_src_nocomments(file_name=zeppelin_joined+crowdsale_joined)
# raw_lines = remove_import_lines(raw_lines)

In [31]:
raw_lines

['\n',
 '\n',
 'contract Crowdsale {\n',
 '  using SafeMath for uint256;\n',
 '\n',
 '  MintableToken public token;\n',
 '\n',
 '  uint256 public startTime;\n',
 '  uint256 public endTime;\n',
 '\n',
 '  address public wallet;\n',
 '\n',
 '  uint256 public rate;\n',
 '\n',
 '  uint256 public weiRaised;\n',
 '\n',
 '  event TokenPurchase(address indexed purchaser, address indexed beneficiary, uint256 value, uint256 amount);\n',
 '\n',
 '\n',
 '  function Crowdsale(uint256 _startTime, uint256 _endTime, uint256 _rate, address _wallet) public {\n',
 '    require(_startTime >= now);\n',
 '    require(_endTime >= _startTime);\n',
 '    require(_rate > 0);\n',
 '    require(_wallet != address(0));\n',
 '\n',
 '    token = createTokenContract();\n',
 '    startTime = _startTime;\n',
 '    endTime = _endTime;\n',
 '    rate = _rate;\n',
 '    wallet = _wallet;\n',
 '  }\n',
 '\n',
 '  function createTokenContract() internal returns (MintableToken) {\n',
 '    return new MintableToken();\n',
 ' 

In [19]:
for raw_line in raw_lines:
    print(raw_line.split())

[]
[]
['contract', 'Crowdsale', '{']
['using', 'SafeMath', 'for', 'uint256;']
[]
['MintableToken', 'public', 'token;']
[]
['uint256', 'public', 'startTime;']
['uint256', 'public', 'endTime;']
[]
['address', 'public', 'wallet;']
[]
['uint256', 'public', 'rate;']
[]
['uint256', 'public', 'weiRaised;']
[]
['event', 'TokenPurchase(address', 'indexed', 'purchaser,', 'address', 'indexed', 'beneficiary,', 'uint256', 'value,', 'uint256', 'amount);']
[]
[]
['function', 'Crowdsale(uint256', '_startTime,', 'uint256', '_endTime,', 'uint256', '_rate,', 'address', '_wallet)', 'public', '{']
['require(_startTime', '>=', 'now);']
['require(_endTime', '>=', '_startTime);']
['require(_rate', '>', '0);']
['require(_wallet', '!=', 'address(0));']
[]
['token', '=', 'createTokenContract();']
['startTime', '=', '_startTime;']
['endTime', '=', '_endTime;']
['rate', '=', '_rate;']
['wallet', '=', '_wallet;']
['}']
[]
['function', 'createTokenContract()', 'internal', 'returns', '(MintableToken)', '{']
['return'

In [22]:
# would need to use hand-crafted regex to split as wanted.
# TODO: implement if tokenize not desirable.

In [24]:
# approach using tokenize


In [32]:
tokens_dict = {
    'line': [],
    'type': [],
    'value': [],
}
tokens_list = []
with open(zeppelin_joined+crowdsale_joined, 'r') as file:
    token_generator = tokenize.generate_tokens(file.readline)
    for type, token, (srow, scol), (erow, ecol), line in token_generator:
        tokens_dict['line'].append(srow)        
        tokens_dict['type'].append(tokenize.tok_name[type])        
        tokens_dict['value'].append(repr(token))
        tokens_list.append([srow, tokenize.tok_name[type], repr(token)])
        
tokens = pd.DataFrame.from_dict(tokens_dict)

In [33]:
tokens

,line,type,value
0,1,NL,'\n'
1,2,NL,'\n'
2,3,NAME,'contract'
3,3,NAME,'Crowdsale'
4,3,OP,'{'
5,3,NL,'\n'
6,4,NAME,'using'
7,4,NAME,'SafeMath'
8,4,NAME,'for'
9,4,NAME,'uint256'


In [34]:
# token_types, type_counts = np.unique(tokens['type'], return_counts=True)
# print(token_types,type_counts)

token_type_freq = pd.Series.value_counts(tokens.type, sort=True)
print(token_type_freq)

NAME         152
OP           148
NL            72
NUMBER         4
NEWLINE        1
ENDMARKER      1
Name: type, dtype: int64


In [35]:
token_freq = pd.Series.value_counts(tokens.value, sort=True)
print(token_freq[:20])

'\n'             73
';'              32
'('              29
')'              29
'uint256'        12
'='              10
','              10
'public'          9
'.'               9
'}'               8
'{'               8
'function'        7
'address'         7
'require'         6
'msg'             5
'beneficiary'     5
'&'               4
'0'               4
'value'           4
'bool'            4
Name: value, dtype: int64


In [36]:
op_freq = pd.Series.value_counts(tokens[tokens.type == 'OP'].value, sort=True)
print(op_freq[:20])

';'     32
'('     29
')'     29
'='     10
','     10
'.'      9
'}'      8
'{'      8
'&'      4
'!='     3
'>='     3
'>'      2
'<='     1
Name: value, dtype: int64


In [37]:
name_freq = pd.Series.value_counts(tokens[tokens.type == 'NAME'].value, sort=True)
print(name_freq[:20])

'uint256'        12
'public'          9
'function'        7
'address'         7
'require'         6
'beneficiary'     5
'msg'             5
'endTime'         4
'now'             4
'bool'            4
'value'           4
'weiAmount'       4
'_startTime'      4
'_wallet'         3
'weiRaised'       3
'wallet'          3
'rate'            3
'tokens'          3
'return'          3
'returns'         3
Name: value, dtype: int64


In [14]:
## Next steps:

In [ ]:
# 1. write tokenization function that receives a list of paths 
# and does the tokenization and counts for all files in paths.


In [ ]:
# 2. Plot the Data (with matplotlib or port over to R)